<a href="https://colab.research.google.com/github/giocarro/Data_Science_Gio/blob/main/Data_Science_Gio/Tareas/Diamonds_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Cargar Librerias

In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix
from sklearn.decomposition import PCA

In [ ]:
%cd /content/drive/MyDrive/Colab Notebooks

/content/drive/MyDrive/Colab Notebooks


# Definicion de la Matriz de confusion

In [ ]:
def plot_confusion_matrix(cm, labels):
    fig_cm = px.imshow(cm, labels=dict(x="Predicted", y="Actual", color="Count"),
                       x=labels, y=labels, color_continuous_scale='Viridis', text_auto = True,
                       title="Confusion Matrix")
    fig_cm.update_layout(coloraxis_showscale=False)
    fig_cm.show()

# Leyendo el CSV

In [ ]:
df = pd.read_csv('diamonds.csv')
df

,carat,cut,color,clarity,depth,table,price,x,y,z
0,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
1,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31
2,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31
3,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63
4,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75
...,...,...,...,...,...,...,...,...,...,...
53935,0.72,Ideal,D,SI1,60.8,57.0,2757,5.75,5.76,3.50
53936,0.72,Good,D,SI1,63.1,55.0,2757,5.69,5.75,3.61
53937,0.70,Very Good,D,SI1,62.8,60.0,2757,5.66,5.68,3.56
53938,0.86,Premium,H,SI2,61.0,58.0,2757,6.15,6.12,3.74


In [ ]:
df2 = df.copy()

In [ ]:
df2

,carat,cut,color,clarity,depth,table,price,x,y,z
0,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
1,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31
2,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31
3,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63
4,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75
...,...,...,...,...,...,...,...,...,...,...
53935,0.72,Ideal,D,SI1,60.8,57.0,2757,5.75,5.76,3.50
53936,0.72,Good,D,SI1,63.1,55.0,2757,5.69,5.75,3.61
53937,0.70,Very Good,D,SI1,62.8,60.0,2757,5.66,5.68,3.56
53938,0.86,Premium,H,SI2,61.0,58.0,2757,6.15,6.12,3.74


In [ ]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53940 entries, 0 to 53939
Data columns (total 10 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   carat    53940 non-null  float64
 1   cut      53940 non-null  object 
 2   color    53940 non-null  object 
 3   clarity  53940 non-null  object 
 4   depth    53940 non-null  float64
 5   table    53940 non-null  float64
 6   price    53940 non-null  int64  
 7   x        53940 non-null  float64
 8   y        53940 non-null  float64
 9   z        53940 non-null  float64
dtypes: float64(6), int64(1), object(3)
memory usage: 4.1+ MB


In [ ]:
df2['clarity'].value_counts()

SI1     13065
VS2     12258
SI2      9194
VS1      8171
VVS2     5066
VVS1     3655
IF       1790
I1        741
Name: clarity, dtype: int64

In [ ]:
df2['clarity'].nunique()

8

In [ ]:
# This will be the target
# df2['cut'].replace(['Fair', 'Good', 'Very Good', 'Premium', 'Ideal'],
#                         [0, 1, 2, 3, 4], inplace=True)

In [ ]:
df2['color'].replace(['D', 'E', 'F', 'G', 'H', 'I', 'J'],
                        [0, 1, 2, 3, 4, 5, 6], inplace=True)

In [ ]:
df2['clarity'].replace(['I3', 'I2', 'I1', 'SI2', 'SI1', 'VS2', 'VS1', 'VVS2', 'VVS1', 'IF', 'FL'],
                        [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10], inplace=True)

## Cambiando el tipo de algunas columnas a categoricas

In [ ]:
# # Cambiar tipo de columna
# from pandas.api.types import CategoricalDtype

# cut_cat_type = CategoricalDtype(categories = ['Fair', 'Good', 'Very Good', 'Premium', 'Ideal'], ordered = True)
# df['cut'] = df.cut.astype(cut_cat_type)

# color_cat_type = CategoricalDtype(categories = ['D', 'E', 'F', 'G', 'H', 'I', 'J'], ordered = True)
# df['color'] = df.color.astype(color_cat_type)

# clarity_cat_type = CategoricalDtype(categories = ['I3', 'I2', 'I1', 'SI2', 'SI1', 'VS2', 'VS1', 'VVS2', 'VVS1', 'IF', 'FL'], ordered = True)
# df['clarity'] = df.clarity.astype(clarity_cat_type)

# df.head()

In [ ]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53940 entries, 0 to 53939
Data columns (total 10 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   carat    53940 non-null  float64
 1   cut      53940 non-null  object 
 2   color    53940 non-null  int64  
 3   clarity  53940 non-null  int64  
 4   depth    53940 non-null  float64
 5   table    53940 non-null  float64
 6   price    53940 non-null  int64  
 7   x        53940 non-null  float64
 8   y        53940 non-null  float64
 9   z        53940 non-null  float64
dtypes: float64(6), int64(3), object(1)
memory usage: 4.1+ MB


In [ ]:
#Features
X = df2.drop(['cut'], axis=1)
X.head()

,carat,color,clarity,depth,table,price,x,y,z
0,0.23,1,3,61.5,55.0,326,3.95,3.98,2.43
1,0.21,1,4,59.8,61.0,326,3.89,3.84,2.31
2,0.23,1,6,56.9,65.0,327,4.05,4.07,2.31
3,0.29,5,5,62.4,58.0,334,4.20,4.23,2.63
4,0.31,6,3,63.3,58.0,335,4.34,4.35,2.75


In [ ]:
#Target
y = df2['cut']
y

0            Ideal
1          Premium
2             Good
3          Premium
4             Good
           ...    
53935        Ideal
53936         Good
53937    Very Good
53938      Premium
53939        Ideal
Name: cut, Length: 53940, dtype: object

# Grafica de barras

In [ ]:
class_counts = df2['cut'].value_counts()
class_counts
# class_names = breast_cancer.target_names

# print("Class Distribution:")
# for i, count in enumerate(class_counts):
#     class_name = class_names[i]
#     percentage = (count / len(df2)) * 100
#     print(f"{class_name}: {count} ({percentage:.2f}%)")

Ideal        21551
Premium      13791
Very Good    12082
Good          4906
Fair          1610
Name: cut, dtype: int64

In [ ]:
class_counts.index

Index(['Ideal', 'Premium', 'Very Good', 'Good', 'Fair'], dtype='object')

In [ ]:
fig = px.bar(x=class_counts.index, y=class_counts.values, labels={'x': 'Class', 'y': 'Count'}, title='Cut Class Distribution', template = 'plotly_white', text = class_counts.values)
fig.show()

# Matriz de correlacion

In [ ]:
corr_matrix = df2.corr()

fig = px.imshow(corr_matrix, color_continuous_scale = 'RdBu', range_color=[-1,1], text_auto = True, aspect = "auto", labels=dict(color="Correlation"), title='Feature Correlation', height = 800)
fig.update(layout_coloraxis_showscale=False)
fig.show()

<ipython-input-119-5890acbc28f7>:1: FutureWarning:

The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.



In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=7)

In [ ]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Logistic Regression

In [ ]:
lr = LogisticRegression()
lr.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [ ]:
lr_pred = lr.predict(X_test)

In [ ]:
lr_accuracy = accuracy_score(y_test, lr_pred)
print(f"Logistic Regression Accuracy: {lr_accuracy:.2f}")

Logistic Regression Accuracy: 0.40


In [ ]:
lr_precision = precision_score(y_test, lr_pred, average='micro')
print(f"Logistic Regression Precission: {lr_precision:.2f}")

Logistic Regression Precission: 0.40


In [ ]:
lr_recall = recall_score(y_test, lr_pred, average='micro')
print(f"Logistic Regression Recall: {lr_recall:.2f}")

Logistic Regression Recall: 0.40


In [ ]:
lr_f1 = f1_score(y_test, lr_pred, average = 'micro')
print(f"Logistic Regression F1 Score: {lr_f1:.2f}")

Logistic Regression F1 Score: 0.40


In [ ]:
lr_report = classification_report(y_test, lr_pred)
print("Logistic Regression Classification Report:")
print(lr_report)

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Logistic Regression Classification Report:
              precision    recall  f1-score   support

        Fair       0.00      0.00      0.00       324
        Good       0.00      0.00      0.00      1039
       Ideal       0.40      0.93      0.56      4230
     Premium       0.37      0.12      0.18      2770
   Very Good       0.00      0.00      0.00      2425

    accuracy                           0.40     10788
   macro avg       0.15      0.21      0.15     10788
weighted avg       0.25      0.40      0.27     10788



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
df['cut'].unique()

array(['Ideal', 'Premium', 'Good', 'Very Good', 'Fair'], dtype=object)

In [ ]:
lr_cm = confusion_matrix(y_test, lr_pred)
plot_confusion_matrix(lr_cm, df['cut'].unique())

# K-Nearest Neighbors (KNN)

In [ ]:
knn = KNeighborsClassifier()
knn.fit(X_train_scaled, y_train)

KNeighborsClassifier()

In [ ]:
knn_pred = knn.predict(X_test_scaled)

In [ ]:
knn_accuracy = accuracy_score(y_test, knn_pred)
print(f"KNN Accuracy: {knn_accuracy:.2f}")

KNN Accuracy: 0.66


In [ ]:
knn_precision = precision_score(y_test, knn_pred, average='micro')
print(f"KNN Precission: {knn_precision:.2f}")

KNN Precission: 0.66


In [ ]:
knn_recall = recall_score(y_test, knn_pred, average='micro')
print(f"KNN Recall: {knn_recall:.2f}")

KNN Recall: 0.66


In [ ]:
knn_f1 = f1_score(y_test, knn_pred, average='micro')
print(f"KNN F1 Score: {knn_f1:.2f}")

KNN F1 Score: 0.66


In [ ]:
knn_report = classification_report(y_test, knn_pred)
print("KNN Classification Report:")
print(knn_report)

KNN Classification Report:
              precision    recall  f1-score   support

        Fair       0.85      0.73      0.79       324
        Good       0.56      0.52      0.54      1039
       Ideal       0.77      0.90      0.83      4230
     Premium       0.62      0.68      0.65      2770
   Very Good       0.43      0.27      0.33      2425

    accuracy                           0.66     10788
   macro avg       0.65      0.62      0.63     10788
weighted avg       0.64      0.66      0.64     10788



In [ ]:
knn_cm = confusion_matrix(y_test, knn_pred)
plot_confusion_matrix(knn_cm, df['cut'].unique())